# Get global mean data 

from Ben Sanderson's flat10MIP paper analysis and save as csv

In [1]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import glob
import matplotlib.patches as patches
from scipy.stats import linregress

import os

In [2]:
# folder containing data sub folders
bsdataloc='/glade/work/aswann/jupyter_notebooks/esm-deck/'


## Emissions data

In [3]:
# load emissions data

e_flat10=pd.read_csv(bsdataloc +'flat10.csv',header=4)
e_flat10_cdr=pd.read_csv(bsdataloc +'flat10_cdr.csv',header=4)
e_flat10_zec=pd.read_csv(bsdataloc +'flat10_zec.csv',header=4)
e_flat10_nz=pd.read_csv(bsdataloc +'flat10_cdr.csv',header=4)
e_flat10_nz[150:]=0

## Load data

In [4]:
# ESM data
def extd(ts,leng):
    out=np.zeros(leng)
    if leng>len(ts):
        out[:len(ts)]=ts.values
        out[len(ts):]=ts.iloc[-1]
    else:
        out=ts[:leng]
    return out
#mdls=['ACCESS-ESM1-5','CESM2','GFDL-ESM4','GISS','NorESM2-LM','CNRM-ESM2-1','MPI-ESM1-2-LR']
mdls=['ACCESS-ESM1-5','CESM2','GFDL-ESM4','GISS','NorESM2-LM','MPI-ESM1-2-LR','CNRM-ESM2-1','UKESM','UVic-ESCM-2-10','HadCM3LC-Bris']
expt=['esm-pictrl','1pctco2','flat10','flat10-zec','flat10-cdr']
flds=['tas','co2','fgco2','nbp','fCmissing']

ed={}
for m in mdls:
    ed[m]={}
    for e in expt:
        tmpa=[]
        for f in flds:
                print(f)
                fl10=bsdataloc +'ESM_data/'+m+'/'+'flat10'+'_'+f+'_'+m+'.nc'
                fl=bsdataloc +'ESM_data/'+m+'/'+e+'_'+f+'_'+m+'.nc'
                if os.path.exists(fl) and os.path.exists(fl10):
                    otmp=xr.open_dataset(fl)
                    otmp10=xr.open_dataset(fl10)
                    if 'nep' in list(otmp.keys()):
                        otmp=otmp.rename_vars({"nep": "nbp"})
                    if 'nep' in list(otmp10.keys()):
                        otmp10=otmp10.rename_vars({"nep": "nbp"})
                    if 'CO2_GLOB' in list(otmp10.keys()):
                        otmp10=otmp10.rename_vars({"CO2_GLOB": "co2"})
                        otmp=otmp.rename_vars({"CO2_GLOB": "co2"})
                    if 'NBP_GLOB' in list(otmp10.keys()):
                        otmp10=otmp10.rename_vars({"NBP_GLOB": "nbp"})
                        otmp=otmp.rename_vars({"NBP_GLOB": "nbp"})
                    if 'FGCO2_GLOB' in list(otmp10.keys()):
                        otmp10=otmp10.rename_vars({"FGCO2_GLOB": "fgco2"})
                        otmp=otmp.rename_vars({"FGCO2_GLOB": "fgco2"})    
                    if 'TIME' in list(otmp10.coords.keys()):                  
                        otmp10=otmp10.swap_dims({"TIME": "time"}).rename_vars({"TIME": "time"})
                    if 't' in list(otmp10.coords.keys()):                  
                        otmp10=otmp10.swap_dims({"t": "time"}).rename_vars({"t": "time"})
                    if 't' in list(otmp.coords.keys()):                  
                        otmp=otmp.swap_dims({"t": "time"}).rename_vars({"t": "time"})
                    if 'TIME' in list(otmp.coords.keys()):                  
                        otmp=otmp.swap_dims({"TIME": "time"}).rename_vars({"TIME": "time"})
                    if 'TAS_GLOB' in list(otmp10.keys()):                  
                        otmp10=otmp10.rename_vars({"TAS_GLOB": "tas"})
                    if 'TAS_GLOB' in list(otmp.keys()):                  
                        otmp=otmp.rename_vars({"TAS_GLOB": "tas"})
                        
                    if 'temp_mm_1_5m' in list(otmp10.keys()):                  
                        otmp10=otmp10.rename_vars({"temp_mm_1_5m": "tas"})
                    if 'temp_mm_1_5m' in list(otmp.keys()):                  
                        otmp=otmp.rename_vars({"temp_mm_1_5m": "tas"})
                    
                    if 'field1564_mm_hyb' in list(otmp10.keys()):
                        otmp10=otmp10.rename_vars({"field1564_mm_hyb": "co2"})
                        otmp=otmp.rename_vars({"field1564_mm_hyb": "co2"})
                    if 'unknown' in list(otmp10.keys()):
                        otmp10=otmp10.rename_vars({"unknown": "nbp"})
                        otmp=otmp.rename_vars({"unknown": "nbp"})
                   
                    if 'field1560_mm_srf' in list(otmp10.keys()):
                        otmp10=otmp10.rename_vars({"field1560_mm_srf": "fgco2"})
                        otmp=otmp.rename_vars({"field1560_mm_srf": "fgco2"})
                    if 'FCMISSING' in list(otmp.keys()):                  
                        otmp=otmp.rename_vars({"FCMISSING": "fCmissing"})    
                    if 'FCMISSING' in list(otmp10.keys()):                  
                        otmp10=otmp10.rename_vars({"FCMISSING": "fCmissing"})      
                    if '__xarray_dataarray_variable__' in list(otmp10.keys()):                  
                        otmp10=otmp10.rename_vars({"__xarray_dataarray_variable__": f})
                    if '__xarray_dataarray_variable__' in list(otmp.keys()):                  
                        otmp=otmp.rename_vars({"__xarray_dataarray_variable__": f})

                    tmp=otmp.copy(deep=True)
                    tmp10=otmp10.copy(deep=True)
                    if len(tmp.time) > 4800:
                        tmp = tmp.isel(time=slice(0, 4800))
                    if len(tmp10.time) > 4800:
                        tmp10 = tmp10.isel(time=slice(0, 4800))
                    if e=='flat10':
                        if len(tmp.time)<1000:
                            tmp.coords['time']=pd.date_range('1700-01-01', periods=tmp['time'].values.shape[0],freq='Y')
                        else:
                            tmp.coords['time']=pd.date_range('1700-01-01', periods=tmp['time'].values.shape[0],freq='M')
                        tmp=tmp[f]    
                    else: 
                        if len(tmp.time)<1000:
                            tmp10.coords['time']=pd.date_range('1700-01-01', periods=tmp10['time'].values.shape[0],freq='Y')
                            tmp.coords['time']=pd.date_range('1800-01-01', periods=tmp['time'].values.shape[0],freq='Y')
                        else:
                            tmp10.coords['time']=pd.date_range('1700-01-01', periods=tmp10['time'].values.shape[0],freq='M')
                            tmp.coords['time']=pd.date_range('1800-01-01', periods=tmp['time'].values.shape[0],freq='M')
  
                        tmp=xr.concat((tmp10.where(tmp10['time.year'] < 1800, drop=True)[f],tmp[f]),'time')
                    tmp=tmp.groupby('time.year').mean()
                    if len(tmp.shape)==2:
                        tmp=tmp.sel(lat=0).sel(lon=0).drop('lev')
                    if len(tmp.shape)==3:
                        tmp=tmp.sel(lat=0).sel(lon=0).drop('lat').drop('lon')
                    if len(tmp.shape)==4:
                        if 'lev' in list(otmp10.coords.keys()):
                            tmp=tmp.sel(lat=0).sel(lon=0).isel(lev=[0]).drop('lat').drop('lon').drop('lev')
                        else:
                            tmp=tmp.sel(lat=0).sel(lon=0).isel(plev=[0]).drop('lat').drop('lon').drop('plev')
                    
                    if tmp.name=='tas':            
                        t10=tmp[0:20].mean().values
                        d10=tmp[20:40].mean().values-t10
                        t0=t10-d10/2
                        tmp=tmp-t0
                    tmpa.append(tmp)
                    if f=='fgco2' and m=='NorESM2-LM' and e=='flat10':
                     0
                else:
                    print(m+e+f)
                
             
        ed[m][e]=xr.merge(tmpa)
        if len(tmpa)>0:
            ed[m][e]['fco2fos']=ed[m][e]['tas'].copy()
            if e=='flat10':
                    ed[m][e]['fco2fos'].values=extd(e_flat10['Emission Rate (PgC a-1)'],len(ed[m][e]['year']))
            if e=='flat10-cdr':
                    ed[m][e]['fco2fos'].values=extd(e_flat10_cdr['Emission Rate (PgC a-1)'],len(ed[m][e]['year']))
            if e=='flat10-zec':
                    ed[m][e]['fco2fos'].values=extd(e_flat10_zec['Emission Rate (PgC a-1)'],len(ed[m][e]['year']))
            if m=='ACCESS-ESM1-5':
                ed[m][e]['co2']=ed[m][e]['co2']*1e6

            if m=='GFDL-ESM4':
                ed[m][e]['co2']=ed[m][e]['co2'].isel(lev=0)*1e6
                #ed[m][e]['nbp']=ed[m][e]['nbp']*5.1e14/1e12*3600*24*365
                aco2=np.diff(ed[m][e]['co2'][:],append=ed[m][e]['co2'][-1])*2.12
                ln=len(ed[m][e]['fco2fos'])
                ed[m][e]['fgco2']=ed[m][e]['fgco2'][:ln]
                ed[m][e]['nbp']=ed[m][e]['fco2fos'][:ln]-aco2[:ln]-ed[m][e]['fgco2'][:ln]

            if m=='CESM2':
                ed[m][e]['fgco2']=ed[m][e]['fgco2']/1e15
                ed[m][e]['nbp']=ed[m][e]['nbp']/1e15
                
                ed[m][e]['co2']=280+np.cumsum(ed[m][e]['fco2fos']-ed[m][e]['fgco2']-ed[m][e]['nbp'])/2.12
            
                ed[m][e]['co2']=280+np.cumsum(ed[m][e]['fco2fos']-ed[m][e]['fgco2']-ed[m][e]['nbp'])/2.12
            if m=='NorESM2-LM':
                ed[m][e]['co2']=ed[m][e]['co2']*1e6
                #ed[m][e]['nbp']=ed[m][e]['nbp']#*5.1e14/1e12*3600*24*365
                aco2=np.diff(ed[m][e]['co2'][:],append=ed[m][e]['co2'][-1])*2.12
                ed[m][e]['nbp']=ed[m][e]['fco2fos']-aco2-ed[m][e]['fgco2']
                #ed[m][e]['fgco2']=ed[m][e]['fgco2']*5.1e14/1e12*3600*24*365
            if m=='UKESM':
                ed[m][e]['nbp']=ed[m][e]['nbp']*5.1e14/1e12*.29
                #ed[m][e]['nbp']=ed[m][e]['nbp']*5.1e14/1e12*3600*24*365
                #ed[m][e]['fgco2']=ed[m][e]['fgco2']*5.1e14/1e12*3600*24*365*.71
            if m=='CNRM-ESM2-1':
                ed[m][e]['fgco2']=ed[m][e]['fgco2']-ed[m][e]['fCmissing']
                #ed[m][e]['nbp']=ed[m][e]['nbp']*5.1e14/1e12*3600*24*365
                #ed[m][e]['fgco2']=ed[m][e]['fgco2']*5.1e14/1e12*3600*24*365*.71
            if m=='HadCM3LC-Bris':
                0#ed[m][e]['fgco2']=ed[m][e]['fgco2']*5.1e14/1e12*.71
                ed[m][e]['nbp']=ed[m][e]['nbp']
                


ed["GFDL-ESM4"]['flat10']['fgco2'][0] = ed["GFDL-ESM4"]['flat10']['fgco2'][0]-10        # correct first ocean value of GFDL-ESM4
ed["GFDL-ESM4"]['flat10-zec']['fgco2'][0] = ed["GFDL-ESM4"]['flat10-zec']['fgco2'][0]-10        # correct first ocean value of GFDL-ESM4
ed["GFDL-ESM4"]['flat10-cdr']['fgco2'][0] = ed["GFDL-ESM4"]['flat10-cdr']['fgco2'][0]-10        # correct first ocean value of GFDL-ESM4

for i,m in enumerate(mdls):
    print(m)
    try:
        ed[m]['flat10-cdr']['C_atm']=(ed[m]['flat10-cdr']['co2']-ed[m]['flat10']['co2'][0])*2.13
    except:
        print('fail cdr')
    try:
        ed[m]['flat10-zec']['C_atm']=(ed[m]['flat10-zec']['co2']-ed[m]['flat10']['co2'][0])*2.13
    except:
        print('fail zec')
    try:
        ed[m]['flat10']['C_atm']=(ed[m]['flat10']['co2']-ed[m]['flat10']['co2'][0])*2.13

    except:
        print('fail flat')    


t0mat = {}
for i, m in enumerate(mdls):
    # Extract the first 30 years of the temperature timeseries
    x = np.arange(0, 30)  # Years 0 to 29
    y = np.array(ed[m]['flat10']['tas'][:30])  # Corresponding temperature values

    # Perform linear regression to estimate temperature at time zero
    slope, intercept, _, _, _ = linregress(x, y)
    t0mat[m] = intercept  # Use the intercept as the temperature at time zero

    # Adjust the temperature timeseries for each experiment
    ed[m]['flat10-cdr']['tas'] = ed[m]['flat10-cdr']['tas'] - intercept
    ed[m]['flat10-zec']['tas'] = ed[m]['flat10-zec']['tas'] - intercept
    ed[m]['flat10']['tas'] = ed[m]['flat10']['tas'] - intercept

tas
ACCESS-ESM1-5esm-pictrltas
co2
ACCESS-ESM1-5esm-pictrlco2
fgco2
ACCESS-ESM1-5esm-pictrlfgco2
nbp
ACCESS-ESM1-5esm-pictrlnbp
fCmissing
ACCESS-ESM1-5esm-pictrlfCmissing
tas
ACCESS-ESM1-51pctco2tas
co2
ACCESS-ESM1-51pctco2co2
fgco2
ACCESS-ESM1-51pctco2fgco2
nbp
ACCESS-ESM1-51pctco2nbp
fCmissing
ACCESS-ESM1-51pctco2fCmissing
tas


/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/xarray/coding/times.py:1133: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/xarray/core/indexing.py:511: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(self.get_duck_array(), dtype=dtype)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/xarray/coding/times.py:1133: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/glade/u/apps/o

co2
fgco2
nbp


/glade/derecho/scratch/aswann/tmp/ipykernel_66784/2197057741.py:102: DeprecationWarning: dropping variables using `drop` is deprecated; use drop_vars.
  tmp=tmp.sel(lat=0).sel(lon=0).drop('lat').drop('lon')
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/xarray/coding/times.py:1133: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  dtype = _decode_cf_datetime_dtype(data, units, calendar, self.use_cftime)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/xarray/core/indexing.py:511: SerializationWarning: Unable to decode time axis into full numpy.datetime64 objects, continuing using cftime.datetime objects instead, reason: dates out of range
  return np.asarray(self.get_duck_array(), dtype=dtype)
/glade/u/apps/opt/conda/envs/npl-2025a/lib/python3.12/site-packages/xarray/coding/times.py:1133: SerializationWarning: Unable to decode time ax

fCmissing
ACCESS-ESM1-5flat10fCmissing
tas
co2


...

co2
fgco2
nbp
fCmissing
HadCM3LC-Brisflat10fCmissing
tas
co2


/glade/derecho/scratch/aswann/tmp/ipykernel_66784/2197057741.py:94: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  tmp10.coords['time']=pd.date_range('1700-01-01', periods=tmp10['time'].values.shape[0],freq='M')
/glade/derecho/scratch/aswann/tmp/ipykernel_66784/2197057741.py:95: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  tmp.coords['time']=pd.date_range('1800-01-01', periods=tmp['time'].values.shape[0],freq='M')
/glade/derecho/scratch/aswann/tmp/ipykernel_66784/2197057741.py:94: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  tmp10.coords['time']=pd.date_range('1700-01-01', periods=tmp10['time'].values.shape[0],freq='M')
/glade/derecho/scratch/aswann/tmp/ipykernel_66784/2197057741.py:95: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  tmp.coords['time']=pd.date_range('180

fgco2
nbp
fCmissing
HadCM3LC-Brisflat10-zecfCmissing
tas
co2


/glade/derecho/scratch/aswann/tmp/ipykernel_66784/2197057741.py:94: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  tmp10.coords['time']=pd.date_range('1700-01-01', periods=tmp10['time'].values.shape[0],freq='M')
/glade/derecho/scratch/aswann/tmp/ipykernel_66784/2197057741.py:95: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  tmp.coords['time']=pd.date_range('1800-01-01', periods=tmp['time'].values.shape[0],freq='M')
/glade/derecho/scratch/aswann/tmp/ipykernel_66784/2197057741.py:94: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  tmp10.coords['time']=pd.date_range('1700-01-01', periods=tmp10['time'].values.shape[0],freq='M')
/glade/derecho/scratch/aswann/tmp/ipykernel_66784/2197057741.py:95: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  tmp.coords['time']=pd.date_range('180

fgco2
nbp
fCmissing
HadCM3LC-Brisflat10-cdrfCmissing
ACCESS-ESM1-5
CESM2
GFDL-ESM4
GISS
NorESM2-LM
MPI-ESM1-2-LR
CNRM-ESM2-1
UKESM
UVic-ESCM-2-10
HadCM3LC-Bris


In [5]:
ed

{'ACCESS-ESM1-5': {'esm-pictrl': <xarray.Dataset> Size: 0B
  Dimensions:  ()
  Data variables:
      *empty*,
  '1pctco2': <xarray.Dataset> Size: 0B
  Dimensions:  ()
  Data variables:
      *empty*,
  'flat10': <xarray.Dataset> Size: 7kB
  Dimensions:  (year: 200)
  Coordinates:
      height   float64 8B ...
    * year     (year) int64 2kB 1700 1701 1702 1703 1704 ... 1896 1897 1898 1899
      depth    float64 8B ...
  Data variables:
      tas      (year) float32 800B -0.1117 -0.02591 -0.02414 ... 3.526 3.575 3.661
      co2      (year) float32 800B 289.3 293.4 297.0 300.8 ... 827.1 831.4 835.5
      fgco2    (year) float32 800B 0.3446 0.8251 1.258 1.678 ... 3.367 3.265 3.259
      nbp      (year) float32 800B 0.9857 0.943 1.121 ... -2.694 -1.8 -2.076
      fco2fos  (year) float64 2kB 10.0 10.0 10.0 10.0 10.0 ... 10.0 10.0 10.0 10.0
      C_atm    (year) float32 800B 0.0 8.612 16.35 ... 1.155e+03 1.163e+03,
  'flat10-zec': <xarray.Dataset> Size: 12kB
  Dimensions:  (year: 320)
  Coor

## can save to csv files like this

In [9]:
#----- flat10

expt_list=['flat10','flat10-zec','flat10-cdr']

for ex in expt_list:
    # Initialize empty dictionaries to store data for each model
    data_a = {}
    data_d = {}
    data_g = {}
    data_j = {}
    
    # Iterate over models and dynamically handle timeseries lengths
    for model in mdls:
        # Get the length of the timeseries for the current model
        length = len(ed[model][ex]['tas'])
    
        # Extract data for the available length
        data_a[model] = ed[model][ex]['tas'].values[:length]
        data_d[model] = ed[model][ex]['co2'].values[:length]
        data_g[model] = np.cumsum(ed[model][ex]['nbp'].values[:length])
        data_j[model] = np.cumsum(ed[model][ex]['fgco2'].values[:length])
    
    # Convert dictionaries to DataFrames
    df_a = pd.DataFrame.from_dict(data_a, orient='index').transpose()
    df_d = pd.DataFrame.from_dict(data_d, orient='index').transpose()
    df_g = pd.DataFrame.from_dict(data_g, orient='index').transpose()
    df_j = pd.DataFrame.from_dict(data_j, orient='index').transpose()
    
    # Save the DataFrames to CSV files
    df_a.to_csv("globalmean_output/temperatures_" +ex +".csv", index=False)
    df_d.to_csv("globalmean_output/atmospheric_co2_" +ex +".csv", index=False)
    df_g.to_csv("globalmean_output/land_carbon_sink_" +ex +".csv", index=False)
    df_j.to_csv("globalmean_output/ocean_carbon_sink_" +ex +".csv", index=False)


